In [11]:
'''
参考文章 QONN in pennylanes
'''
import numpy as np
import pennylane as qml
from pennylane.operation import Operation

In [47]:
class RBS(Operation):
    num_params = 1
    num_wires = 2
    par_domain = "R"

    def __init__(self, theta, wires):
        self.theta = theta
        super().__init__(theta, wires=wires)

    def expand(self):
    # expand 定义了此量子操作的展开形式
        qml.Hadamard(wires=self.wires[0])
        qml.Hadamard(wires=self.wires[1])
        qml.CZ(wires=[self.wires[0],self.wires[1]])
        qml.RY(self.theta/2, wires=self.wires[0])
        qml.RY(-self.theta/2, wires=self.wires[1])
        qml.CZ(wires=[self.wires[0],self.wires[1]])
        qml.Hadamard(wires=self.wires[0])
        qml.Hadamard(wires=self.wires[1])

In [49]:
dev = qml.device('default.qubit', wires=2)
@qml.qnode(dev)
def circuit():
    qml.QubitStateVector([0,1,0,0]
                        ,wires=[0,1])
    RBS(np.pi, [0,1])
    return qml.state()
circuit()

AttributeError: 'NoneType' object has no attribute 'samples_computational_basis'

# 使用矩阵直接生成RBS

In [46]:
# plan B,使用方程生成操作
def my_RBS(params, on_wire):
    """定义一个自定义的参数化二比特门"""
    qml.QubitUnitary(np.array([[1, 0, 0, 0],
                               [0, np.cos(params), np.sin(params), 0],
                               [0, -np.sin(params), np.cos(params), 0],
                               [0, 0, 0, 1]]),
                     wires=on_wire)


In [45]:
dev = qml.device('default.qubit', wires=4)
@qml.qnode(dev)
def circuit(in_state, params):
    qml.QubitStateVector(in_state
                        ,wires=[0,1,2,3])
    my_RBS(params[0], [0,1])
    my_RBS(params[1], [1,2])
    my_RBS(params[2], [0,1])
    my_RBS(params[3], [2,3])
    my_RBS(params[4], [1,2])
    return qml.probs(wires=[2,3])

theta = [1,1,1,1,1]
in_state = np.zeros(2**4)
in_state[1] = 1.0
print(circuit(in_state,theta))
print(qml.draw(circuit)(in_state,theta))

[0.50136797 0.29192658 0.20670545 0.        ]
0: ─╭QubitStateVector(M0)─╭U(M1)────────╭U(M1)────────┤       
1: ─├QubitStateVector(M0)─╰U(M1)─╭U(M1)─╰U(M1)─╭U(M1)─┤       
2: ─├QubitStateVector(M0)────────╰U(M1)─╭U(M1)─╰U(M1)─┤ ╭Probs
3: ─╰QubitStateVector(M0)───────────────╰U(M1)────────┤ ╰Probs
